In [ ]:
import numpy as np
# графики
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
# прогресс-бар
from tqdm.auto import tqdm
# работа с датасетом CIFAR-10
from torchvision.datasets import CIFAR10
# трансформации картинок
from torchvision.transforms import Compose, Lambda, ToTensor
# подготовка батчей данных
from torch.utils.data import DataLoader
# метрики качества классификации
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# необходимая трансформация - датасет возвращает объекты PIL.Image, а нам нужно работать с матрицами
# и заодно сразу вытянем картинку в вектор (flatten - превращает матрицу в плоский вектор)
transform = Compose([
    ToTensor(),
    Lambda(lambda x: x.flatten()),
])
# загрузим датасет (обучающую и тестовую часть)
trainset = CIFAR10(root='.', train=True, download=True, transform=transform)
testset = CIFAR10(root='.', train=False, download=True, transform=transform)

In [ ]:
# классы в нашем датасете
print(trainset.classes)

# Код однослойного перцептрона (код с семинара с изменениями)

In [ ]:
class OneLayerNet:
    
    def __init__(self, in_d, out_d):
        # in_d - размерность входа сети, out_d - размерность выхода сети (количество классов)
        # случайная инициализация весов сети
        self.W = 1e-3 * np.random.randn(in_d, out_d)
        self.bias = np.zeros(out_d)
        print(f'Число параметров в сети: {self.W.size + self.bias.size}')
    
    def softmax(self, scores):
        # функция softmax для превращения оценок в вероятности классов
        # применяется на последнем слое сети
        shift_scores = scores - np.max(scores, axis=1).reshape(-1, 1)
        exp_scores = np.exp(shift_scores)
        sum_exp_scores = np.sum(exp_scores, axis=1).reshape(-1, 1)
        return exp_scores / sum_exp_scores

    def forward(self, X_batch):
        # вычисляет ответ сети для пакета данных, возвращает вероятности классов для каждого входящего примера
        # добавили смещение
        out = X_batch @ self.W + self.bias
        return self.softmax(out)
    
    def predict(self, X_batch):
        # возвращает индекс самого вероятного по мнению сети класса для каждого примера
        # а также вероятности классов
        y_pred = self.forward(X_batch)
        return np.argmax(y_pred, axis=1), y_pred
    
    def backward(self, X_batch, y_batch, reg):
        # рассчитывает значение функции ошибки и ее градиент по весам для применения в градиентном спуске
        # словарь, в который положим градиенты по параметрам
        grads = {}
        batch_size = X_batch.shape[0]
        # получаем ответы сети на данном шаге
        y_pred = self.forward(X_batch)
        # в качестве функции ошибки для задачи классификации используем кросс-энтропию
        loss = -np.sum(np.log(y_pred[range(batch_size), y_batch]))
        # регуляризация
        loss = loss / batch_size + reg * np.sum(self.W ** 2)
        # расчет градиента выводится с помощью математики для данного конкретного типа сети
        y_pred[range(batch_size), y_batch] -= 1
        grads['W'] = X_batch.T @ y_pred
        # регуляризация
        grads['W'] = grads['W'] / batch_size + 2 * reg * self.W
        # bias (смещение)
        grads['bias'] = np.mean(y_pred, axis=0)
        return loss, grads
    
    def fit_batch(self, X_batch, y_batch, lr, reg):
        # обучение сети на одном батче, градиентный спуск
        loss, grads = self.backward(X_batch, y_batch, reg)
        # итерация градиентного спуска
        self.W -= lr * grads['W']
        self.bias -= lr * grads['bias']
        return loss

## Вспомогательные функции, пригодятся и для двухслойной сети

In [ ]:
# функция применения сети к датасету и подсчета метрик
def eval_model(model, dataset, batch_size):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    # контейнеры для предсказаний сети и правильных меток
    predictions, ground_truth = [], []
    val_loss = 0
    for images_batch, labels_batch in tqdm(dataloader, desc='Evaluation', leave=False):
        # конвертация из тензоров в матрицы numpy
        images_batch = images_batch.numpy()
        labels_batch = labels_batch.numpy()
        # у последнего батча может быть несовпадающий размер
        bs = images_batch.shape[0]
        # предсказания модели
        y_pred, y_probs = model.predict(images_batch)
        # используем предсказанные вероятности для расчета функции ошибки (кросс-энтропии)
        val_loss -= np.sum(np.log(y_probs[range(bs), labels_batch])) / bs
        predictions.append(y_pred)
        ground_truth.append(labels_batch)
    val_loss /= len(dataloader)
    predictions = np.concatenate(predictions)
    ground_truth = np.concatenate(ground_truth)
    # предсказанные метки используем для расчета классификационных метрик
    accuracy = accuracy_score(ground_truth, predictions)
    report = classification_report(ground_truth, predictions, target_names=dataset.classes)
    return val_loss, accuracy, report

In [ ]:
# функция обучения сети на датасете
def train_model(model, trainset, testset, batch_size, epochs, lr, reg):
    train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
    # словарь, в который будем складывать метрики чтобы потом нарисовать графики
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_acc': [],
    }
    # немного другой способ отрисовывания прогресс-баров
    with tqdm(total=epochs, desc='Epoch') as epoch_pbar:
        for _ in range(epochs):
            running_loss = 0
            with tqdm(total=len(train_loader), desc='Batch', leave=False) as batch_pbar:
                for idx, (images_batch, labels_batch) in enumerate(train_loader):
                    # конвертация из тензоров в матрицы numpy
                    images_batch = images_batch.numpy()
                    labels_batch = labels_batch.numpy()
                    loss = model.fit_batch(images_batch, labels_batch, lr, reg)
                    # для вывода статистики
                    running_loss += loss
                    # обновим вложенный прогресс-бар
                    batch_pbar.set_postfix(loss=running_loss / (idx + 1))
                    batch_pbar.update()
            running_loss /= len(train_loader)
            # получим метрики на валидации для этой эпохи
            val_loss, val_acc, _ = eval_model(model, testset, batch_size)
            history['train_loss'].append(running_loss)
            history['val_loss'].append(val_loss)
            history['val_acc'].append(val_acc)
            # обновим прогресс-бар эпох
            epoch_pbar.set_postfix(loss=running_loss, val_loss=val_loss, val_acc=val_acc)
            epoch_pbar.update()
    return history

In [ ]:
def plot_metrics(history):
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    n_epochs = len(history['train_loss'])
    axes[0].plot(range(n_epochs), history['train_loss'], label='train_loss')
    axes[0].plot(range(n_epochs), history['val_loss'], label='val_loss')
    axes[0].set_title('Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].legend()

    axes[1].plot(range(n_epochs), history['val_acc'], label='val_acc')
    axes[1].set_title('Accuracy')
    axes[1].set_xlabel('Epoch')
    axes[1].legend()

    plt.show()

In [ ]:
# функция визуализации выученных весов нейронной сети для каждого класса
def visualize_weights(weights_matrix, class_names):
    weights_matrix = weights_matrix.copy()
    # превратим веса относительно каждого класса в набор картинок 32x32x3
    weights_matrix = weights_matrix.reshape(3, 32, 32, 10)
    # переставим каналы в правильном порядке
    weights_matrix = np.moveaxis(weights_matrix, 0, -2)
    # для нормировки узнаем минимальный и максимальный веса
    w_min, w_max = weights_matrix.min(), weights_matrix.max()
    fig, axes = plt.subplots(2, 5, figsize=(16, 8))
    axes = [ax for ax_row in axes for ax in ax_row]
    for idx, ax in enumerate(axes):
        # отмасштабируем веса в отрезок [0, 255]
        w_img = 255 * (weights_matrix[:, :, :, idx].squeeze() - w_min) / (w_max - w_min)
        w_img = w_img.astype('uint8')
        ax.imshow(w_img)
        ax.grid(False)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        ax.set_title(class_names[idx])
    plt.show()

# Задание 1

Вам нужно провести эксперименты по обучению однослойной сети. Постарайтесь получить максимальное качество (accuracy) на тестовой выборке. Экспериментируйте с количеством эпох, batch_size, шагом обучения и коэффициентом регуляризации (reg, дробное число >= 0). По итогам экспериментов зафиксируйте параметры с которыми получилось наилучшее качество и соответствующие графики. Попробуйте описать, как каждый из изменяемых параметров влияет на процесс обучения.

*Хороший результат для однослойной сети - 40 ~ 45% accuracy*

In [ ]:
model = ...
history = ...

In [ ]:
# график метрик
plot_metrics(history)

In [ ]:
# финальные метрики
_, val_acc, val_report = eval_model(model, testset, batch_size=1024)
print(f'Accuracy: {val_acc:.2%}')
print(val_report)

*ваш текст здесь*

# Задание 2

Возьмите свою лучшую модель из первого задания и визуализируйте ее веса с помощью функции ниже. Попробуйте предположить, что на них можно увидеть.

In [ ]:
visualize_weights(model.W, trainset.classes)

*ваш текст здесь*

# Задание 3: Двухслойная сеть

Дополните код в классе двухслойной сети по аналогии с однослойной сетью (для справки используйте материалы лекций и код однослойной сети выше). В качестве функции активации скрытого слоя используйте ReLU. Признаком правильной реализации является постепенное уменьшение train_loss по мере обучения.

In [ ]:
class TwoLayerNet:

    def __init__(self, in_d, h_d, out_d):
        # in_d - размерность входа сети, out_d - размерность выхода сети (количество классов)
        # h_d - размерность скрытого слоя сети
        # случайная инициализация весов сети
        self.W1 = 1e-3 * np.random.randn(in_d, h_d)
        self.bias1 = np.zeros(h_d)
        self.W2 = 1e-3 * np.random.randn(h_d, out_d)
        self.bias2 = np.zeros(out_d)
        print(f'Число параметров в сети: {self.W1.size + self.bias1.size + self.W2.size + self.bias2.size}')
    
    def softmax(self, scores):
        # функция softmax для превращения оценок в вероятности классов
        # применяется на последнем слое сети
        shift_scores = scores - np.max(scores, axis=1).reshape(-1,1)
        exp_scores = np.exp(shift_scores)
        sum_exp_scores = np.sum(exp_scores, axis=1).reshape(-1, 1)
        return exp_scores / sum_exp_scores
    
    def forward(self, X_batch):
        # вычисляет ответ сети для пакета данных, возвращает вероятности классов для каждого входящего примера
        # в качестве функции активации скрытого слоя используйте ReLU (подсказка - вам может помочь функция np.maximum)
        # h - активации скрытого слоя, они нужны методу backward() для правильного расчета градиента
        # ваш код здесь
        out1 = ... + self.bias1
        # h = ReLU, примененная к out1 - выход первого слоя сети
        h = ...
        # умножаем h на соответствующие веса и прибавляем bias
        out2 = ... + self.bias2
        return self.softmax(out2), h
    
    def predict(self, X_batch):
        # метод возвращает индекс самого вероятного по мнению сети класса для каждого примера
        # а также вероятности классов
        y_pred, _ = self.forward(X_batch)
        return np.argmax(y_pred, axis=1), y_pred
    
    def backward(self, X_batch, y_batch, reg):
        # метод рассчитывает значение функции ошибки и ее градиент по весам для применения в градиентном спуске
        batch_size = X_batch.shape[0]
        # получаем ответы сети на данном шаге и активации скрытого слоя
        y_pred, h = self.forward(X_batch)
        # в качестве функции ошибки для задачи классификации используем кросс-энтропию
        loss = -np.sum(np.log(y_pred[range(batch_size), y_batch])) / batch_size
        loss += 0.5 * reg * (np.sum(self.W1 ** 2) + np.sum(self.W2 ** 2))
        # расчет градиента выводится с помощью математики для данного конкретного типа сети
        # градиенты для каждой матрицы весов положим в словарь
        grads = {}
        y_pred[range(batch_size), y_batch] -= 1
        grads['W2'] = h.T @ y_pred
        grads['W2'] = grads['W2'] / batch_size + reg * self.W2
        grads['bias2'] = np.mean(y_pred, axis=0)

        dh = y_pred @ self.W2.T
        dh_relu = (h > 0) * dh
        grads['W1'] = X_batch.T @ dh_relu 
        grads['W1'] = grads['W1'] / batch_size + reg * self.W1
        grads['bias1'] = np.mean(dh_relu, axis=0)
        return loss, grads
    
    def fit_batch(self, X_batch, y_batch, lr, reg):
        # обучение сети на одном батче, градиентный спуск
        loss, grads = self.backward(X_batch, y_batch, reg)
        # итерация градиентного спуска
        # ваш код здесь
        self.W1 = ...
        self.bias1 = ...
        self.W2 = ...
        self.bias2 = ...
        return loss

# Задание 4

Теперь проведите эксприменты с двухслойной сетью. У вас появляется новый параметр - количество нейронов в скрытом слое. Замечание: параметры, подобранные для однослойной сети не будут оптимальными для двухслойной. Если ваша сеть сильно переобучается - либо уменьшайте количество нейронов, либо повышайте параметр reg, если сеть практически не обучается - попробуйте увеличить learning rate или уменьшить reg.

Опишите результаты экспериментов и зафиксируйте параметры и графики лучшей модели.

In [ ]:
model = ...
history = ...

In [ ]:
plot_metrics(history)

*Для хорошо обученной двухслойной сети можно получить accuracy > 50%. При этом двухслойная сеть может сходиться сильно медленнее однослойной.*

*ваш текст здесь*